# Alcohol and Traffic Accidents

## Introduction

In this notebook I will show the effect that drunk driving has on collisions in California as well as showing the areas in which drunk driving is most prevelent.

In [ ]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import plotly.express as px
import json
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
import numpy as np

In [ ]:
con = sqlite3.connect('../input/california-data/switrs.sqlite')
county_populations = pd.read_csv('../input/county-populations/csvData.csv')
countys = json.load(open('../input/geojson/us-county-boundaries.geojson', 'r'))

## Why target drunk driving?

With so many different collision causing factors within this dataset, you may be wondering why I am focusing my attention on drunk driving. 
In this section, I will show the severity of the impact inebriation has on collisions.

In [ ]:
all_data = pd.read_sql_query('''
                  SELECT collision_severity, party_sobriety
                  FROM collisions
                  INNER JOIN parties\
                  ON collisions.case_id = parties.case_id\
                  WHERE '2020-01-01' > collision_date AND\
                  collision_date > '2019-01-01'\
                  AND party_type = 'driver'\
                  AND at_fault = 1
               ''', con)

In [ ]:
drunk_driver_fatality = len(all_data[(all_data["collision_severity"] == 'fatal') & (all_data["party_sobriety"] == 'had been drinking, under influence')])
driver_fatality = len(all_data[(all_data["collision_severity"] == 'fatal')])

drunk_driver_collision = len(all_data[(all_data["collision_severity"] != 'fatal') & (all_data["party_sobriety"] == 'had been drinking, under influence')])
driver_collision = len(all_data[(all_data["collision_severity"] != 'fatal')])


In [ ]:
fatality_data = [driver_fatality - drunk_driver_fatality, drunk_driver_fatality]
collision_data = [driver_collision - drunk_driver_collision, drunk_driver_collision]

fatality_labels = ['Fatalities caused by sober drivers', 'Fatilties caused by drunk drivers']
collision_labels = ['Collisions caused by sober drivers', 'Collisions caused by drunk drivers']
sizes = fatality_data

explode = (0, 0.1)  
fig1, ax1 = plt.subplots(1,2)
ax1[1].pie(fatality_data, explode=explode, labels=fatality_labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 10})
ax1[1].set_title('Driver Caused Fatalites in California (2019 - 2020)', y = 1.05, x = 0.55, fontweight="bold")
ax1[0].pie(collision_data, explode=explode, labels=collision_labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 10})
ax1[0].set_title('Driver Caused Collisions in California (2019 - 2020)', y = 1.05, x = 0.55, fontweight="bold")
fig1.set_size_inches(11, 8)
fig1.tight_layout()
fig1.show()

We see here we have two pie charts. One showing the collisions caused by drunk driving and the other showing the fatalities caused by drunk driving. 
From the charts we can see that while the total number of collisions caused by drunk driving is relitively low, collisions resulting in fatalities is greatly impacted by drunk driving.


# Countys in California in which drunk driving is most prevelent

The best way to try and reduce drunk driving incidents is by targeting the people who do it. We will do this firstly by analysing in which counties in California collisions caused by intoxication happens most.

In [ ]:
new_data = pd.read_sql_query('''
                          SELECT
                          county_location, party_sobriety 
                          FROM collisions
                          INNER JOIN parties
                          ON parties.case_id = collisions.case_id
                          WHERE '2020-01-01' > collision_date
                          AND collision_date > '2019-01-01'
                          AND parties.party_type = 'driver'
                          AND parties.at_fault = 1
                         ''', con)

In [ ]:
def county_groups(dataframe):
    sober_filter = dataframe['party_sobriety'] != 'had been drinking, under influence'
    sober_data = dataframe.loc[sober_filter]['county_location'].value_counts()\
    .rename_axis('county_name').reset_index(name='number_of_sober_collisions')
    drunk_filter = dataframe['party_sobriety'] == 'had been drinking, under influence'
    drunk_data = dataframe.loc[drunk_filter]['county_location'].value_counts()\
    .rename_axis('county_name').reset_index(name='number_of_drunk_collisions')
    return pd.merge(sober_data, drunk_data, on = 'county_name')



In [ ]:
collisions = county_groups(new_data)

In [ ]:
dic = {}
for feature in countys['features']:
    feature['id'] = feature['properties']['countyfp']
    dic[(feature['properties']['name']).lower()] = feature['id']

collisions['id'] = collisions['county_name'].apply(lambda x: dic[x.lower()])



collisions['Proportion_of_drunk_accidents (%)'] = round(collisions['number_of_drunk_collisions'] / collisions['number_of_sober_collisions'] * 100, 2)

In [ ]:
fig = px.choropleth_mapbox(collisions, locations = 'id',
                    geojson = countys, color = 'Proportion_of_drunk_accidents (%)',
                    hover_name = 'county_name', 
                    hover_data = {'Proportion_of_drunk_accidents (%)':True, 'id':False},
                    mapbox_style = 'carto-positron', center = {'lat': 37.7, 'lon': -122},
                    zoom = 4, opacity = 0.7)

In [ ]:
fig.show()

As we can see from the map the proportio of drunk driving against all collisions varies widely throughout the state. What can be seen very clearly is that more rural counties such as Lake County have a much higher proportion of drunk driving cases than densly populated counties such as Los Angeles. This may be due to the lack of public transport in rural areas meaning people are more likely to drive even if they have had been drinking.

## Age groups in which drunk driving is most prevelent

Now that we've seen the Counties where drunk driving happens the most, we will focus on what age groups to target within these counties. 

In [ ]:
drunk_age_group_query = f'''
                           SELECT Count(parties.party_age) AS 'Age_Groups', 
                           CASE WHEN parties.party_age >= 15 
                           AND parties.party_age < 18 THEN '15 - 17'
                           WHEN parties.party_age >= 18 
                           AND parties.party_age < 24 THEN '18 - 24'
                           WHEN parties.party_age >= 24 
                           AND parties.party_age < 35 THEN '25 - 35'
                           WHEN parties.party_age >= 35 
                           AND parties.party_age < 50 THEN '36 - 50'
                           ELSE 'Over 50'
                           END AS Ages_groups
                           FROM parties
                           INNER JOIN collisions 
                           ON collisions.case_id = parties.case_id
                           WHERE parties.party_type = 'driver'
                           AND parties.party_sobriety = 
                           'had been drinking, under influence'
                           AND '2020-01-01' > collisions.collision_date 
                           AND collisions.collision_date > '2019-01-01'
                           AND parties.at_fault = 1
                           GROUP BY Ages_groups
                       '''

all_age_group_query = f'''
                           SELECT Count(parties.party_age) AS 'Age_Groups', 
                           CASE WHEN parties.party_age >= 15 
                           AND parties.party_age < 18 THEN '15 - 17'
                           WHEN parties.party_age >= 18 
                           AND parties.party_age < 24 THEN '18 - 24'
                           WHEN parties.party_age >= 24 
                           AND parties.party_age < 35 THEN '25 - 35'
                           WHEN parties.party_age >= 35 
                           AND parties.party_age < 50 THEN '36 - 50'
                           ELSE 'Over 50'
                           END AS Ages_groups
                           FROM parties
                           INNER JOIN collisions 
                           ON collisions.case_id = parties.case_id
                           WHERE parties.party_type = 'driver'
                           AND '2020-01-01' > collisions.collision_date AND 
                           collisions.collision_date > '2019-01-01'
                           AND parties.at_fault = 1
                           GROUP BY Ages_groups
                       '''

In [ ]:
drunk_age_groups = pd.read_sql_query(drunk_age_group_query, con)
all_age_groups = pd.read_sql_query(all_age_group_query, con)

In [ ]:
full_dataset = pd.merge(drunk_age_groups, all_age_groups, on = 'Ages_groups')
full_dataset['Percentage_of_drunk_collisions'] = (full_dataset['Age_Groups_x'] / full_dataset['Age_Groups_y']) * 100
me = full_dataset[['Ages_groups', 'Percentage_of_drunk_collisions']].set_index('Ages_groups')

In [ ]:
fig, ax = plt.subplots()

bars = ax.bar(
    x=np.arange(me.size),
    height=me['Percentage_of_drunk_collisions'],
    tick_label=full_dataset['Ages_groups']
)

ax.tick_params(axis='both', which='major', labelsize=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

bar_color = bars[0].get_facecolor()
for bar in bars:
  ax.text(
      bar.get_x() + bar.get_width() / 2,
      bar.get_height() + 0.1,
      round(bar.get_height(), 1),
      horizontalalignment='center',
      color=bar_color,
      weight='bold',
      size = 'xx-large'

  )

ax.set_xlabel('Age Groups', labelpad=15, color='#333333', size = 'xx-large')
ax.set_ylabel('collisions caused by drunk driving (%)', labelpad=15, color='#333333', size = 'xx-large')
ax.set_title('Drunk Driving incidents by age group', pad=15, color='#333333', size = 'xx-large',
             weight='bold')
fig.set_size_inches(16, 8)



fig.show()

From the bar graph above we can see that drunk driving happens most often in the 25 - 35 age group. Because of this, we can conclude that the most appropriate people to target are young professionals.